# Configuration

In [ ]:
# Pre-requisite Configuration
# ------------------------------------------------------------------------------ #
# For testing
# ------------------------------------------------------------------------------ #

%load_ext autoreload
%autoreload 2

import os 
from modules.util.config import get_config_by_id, get_config_global
from modules.util.database import SQLAlchemyClient
from modules.util.helpers import Logger


# ------------------------------------------------------------------------------ #
# Configuration
# ------------------------------------------------------------------------------ #

CONFIG_ID = 'dca-test'
TRANSFORM = get_config_by_id(CONFIG_ID)["load"]["indicator"]
EXTRACTION_DIR = TRANSFORM["directory"]
REPORTS_DIR = f"{EXTRACTION_DIR}/reports"
CONFIG_GLOBAL = get_config_global().get('indicators').get('transform')

# ------------------------------------------------------------------------------ #
# Create directories
# ------------------------------------------------------------------------------ #
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)

# ------------------------------------------------------------------------------ #
# Logger
# ------------------------------------------------------------------------------ #

log = Logger.get_logger(CONFIG_ID)

Logger.blank_line(log)
log.info("** LOAD - INDICATORS **")
Logger.blank_line(log)

log.info(f"Extraction Directory: {EXTRACTION_DIR}")
log.info(f"Reports Directory: {REPORTS_DIR}")

db = SQLAlchemyClient(CONFIG_ID)

# 1 - Sync Characteristics with ERP

* ERP characteristics are derived from the *user decision report*.

* The migration tool proposes the ERP characteristics, but user has an option to override the proposal by providing an input.

* The view `V_ERPCharacteristics` contains the list of unique characteristics derived from above.

* Characteristics are **mandatory** to be present (or) created in the ERP system before migrating the indicators.

**Process:**
1. We check the ERP system `[GET]` for characteristics, based on the characteristic name (upper case)

2. If the value already exists in the ERP system, we derive the `CharacInternalID` of the characteristic from the system.

3. In case if the record does not exist, a new characteristic is created `[POST]` in the ERP system 


In [ ]:
# ERP Characteristics
# ------------------------------------------------------------------------------ #

# standard imports
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# custom imports
from modules.util.helpers import convert_dataframe
from modules.erp.s4_clfn_characteristic_srv import ApiCharacteristicHeader
from modules.util.api import APIException
from modules.util.database import (
    SQLAlchemyClient,
    V_ERPCharacteristics,
    ERPCharacteristics,
)

# ------------------------------------------------------------------------------ #
file_tgt = rf'{EXTRACTION_DIR}/1_ERP_Characteristics.parquet'
file_err = rf'{REPORTS_DIR}/1_ERP_Characteristics_Errors.csv'
# ------------------------------------------------------------------------------ #

Logger.blank_line(log)
log.info("Load: ERP Characteristics")
Logger.blank_line(log)

# get characteristics from view
results = db.select(
    model = V_ERPCharacteristics,
    distinct=True,
    orderby=['ERPCharacteristic'],
)

if len(results) > 0:
    # check if characteristic already exist in the ERP system
    df_chars = pd.DataFrame(results)
    df_chars = convert_dataframe(df_chars)
    
    api = ApiCharacteristicHeader(CONFIG_ID)

    response = []
    error = []

    log.info(f"[GET] ERP characteristics based on Characteristic Name")
    # use Threading for parallel processing
    def call_api(char):
        return(api.search_characteristic(char))

    with ThreadPoolExecutor(max_workers=20) as executor:
        future_char = {executor.submit(call_api, char): char for char in df_chars['ERPCharacteristic']}

        for future in as_completed(future_char):
            char = future_char[future]
            try:
                data = future.result()
                if data:
                    response.append(data)
            except APIException as api_e:
                log.error(f"API Exception: {char} - {api_e.status_code} - {api_e.response} - {api_e.endpoint}")
                err = {
                    'characteristic': char,
                    'status_code': api_e.status_code,
                    'response': api_e.response,
                    'endpoint': api_e.endpoint
                }
                error.append(err)

    log.info(f"{len(response)} characteristics fetched.")
    if error:
        log.error(f"{len(error)} characteristics had errors.")

    df_chars = pd.json_normalize(response, sep="_")
    df_chars = convert_dataframe(df_chars)
    df_chars.columns = [col.lstrip('__') if col.startswith('__') else col for col in df_chars.columns]
    df_chars.to_parquet(file_tgt, index=False)
    log.info(f"{file_tgt} generated.")

    if error is not None and len(error)>0:
        df_error = pd.DataFrame(error)
        df_error.to_csv(file_err, index=False)
        log.error(f"{file_err} generated.")
    
    if db.drop_reload:
        db.truncate(ERPCharacteristics)  # truncate all existing data (remove if needed)

    erp_chars = SQLAlchemyClient.dataframe_to_object(df_chars, ERPCharacteristics)
    if erp_chars:
        db.insert_batches(erp_chars)
else:
    log.warning("No characteristics found.")

In [ ]:
# Create Characteristiscs in ERP
# ------------------------------------------------------------------------------ #

from modules.util.database import (
    SQLAlchemyClient,
    V_ERPCharacteristics,
    ERPCharacteristics,
)
from modules.erp.s4_clfn_characteristic_srv import ApiCharacteristicHeader

file_err = rf"{REPORTS_DIR}/2_ERP_Characteristics_Errors.csv"

missing_data = db.select(
    model = V_ERPCharacteristics,
    distinct=True,
    orderby=['ERPCharacteristic'],
    where = [V_ERPCharacteristics.CharcInternalID == None]
)

if len(missing_data) > 0:
    log.info(f"Missing Characteristics in ERP: {len(missing_data)}")
    df_missing = pd.DataFrame(missing_data)
    df_missing = convert_dataframe(df_missing)

    erp_mapping = CONFIG_GLOBAL.get("characteristic").get("erp_mapping")

    df_missing['ERP_DataType'] = df_missing['indicators_datatype'].map(lambda x: erp_mapping.get(x, {}).get('datatype'))
    df_missing['ERP_Length'] = df_missing.apply(
        lambda row: row['indicators_precision'] if row['indicators_precision'] != '0' else erp_mapping.get(row['indicators_datatype'], {}).get('length'), 
        axis=1
    )
    df_missing['ERP_Decimals'] = df_missing.apply(
        lambda row: row['indicators_scale'] if row['indicators_scale'] != '0' else erp_mapping.get(row['indicators_datatype'], {}).get('decimals'), 
        axis=1)

    df_missing['ERP_NegativeFlag'] = df_missing['indicators_datatype'].map(lambda x: erp_mapping.get(x, {}).get('negative'))
    df_missing['ERP_CaseSensitive'] = df_missing['indicators_datatype'].map(lambda x: erp_mapping.get(x, {}).get('case_sensitive'))

    def call_api(char, datatype, length, decimals, description, negative_flag, case_sensitive):
        return(api.create_characteristic(
            char=char, 
            datatype=datatype, 
            length=length, 
            decimals=decimals, 
            description=description,
            negative_flag=negative_flag,
            case_sensitive_flag=case_sensitive))

    api = ApiCharacteristicHeader(CONFIG_ID)
    response = []
    error = []
    log.info(f"[POST] ERP characteristics based on Characteristic Name")

    with ThreadPoolExecutor(max_workers=20) as executor:
        future_char = {executor.submit(call_api, char, dtype, length, decimals, description, negative_flag, case_sensitive): 
                    char for char, dtype, length, decimals, description, negative_flag, case_sensitive in 
                    zip(df_missing['ERPCharacteristic'], df_missing['ERP_DataType'], df_missing['ERP_Length'], df_missing['ERP_Decimals'], df_missing['ERPCharacteristic'], df_missing['ERP_NegativeFlag'], df_missing['ERP_CaseSensitive'])}

        for future in as_completed(future_char):
            char = future_char[future]
            try:
                data = future.result()
                if data:
                    response.append(data)
            except APIException as api_e:
                log.error(f"API Exception: {char} - {api_e.status_code} - {api_e.response} - {api_e.endpoint}")
                err = {
                    'characteristic': char,
                    'status_code': api_e.status_code,
                    'response': api_e.response,
                    'endpoint': api_e.endpoint
                }
                error.append(err)

    log.info(f"{len(response)} characteristics created.")
    if error:
        log.info(f"{len(error)} characteristics had errors.")

    if error is not None and len(error)>0:
        df_error = pd.DataFrame(error)
        df_error.to_csv(file_err, index=False)
        log.error(f"{file_err} generated.")

    df_chars = pd.json_normalize(response, sep="_")
    df_chars = convert_dataframe(df_chars)
    df_chars.columns = [col.lstrip('__') if col.startswith('__') else col for col in df_chars.columns]
    df_chars.drop(columns=['to_CharacteristicDesc_results'], inplace=True)

    if not df_chars.empty:
        erp_chars = SQLAlchemyClient.dataframe_to_object(df_chars, ERPCharacteristics)
        db.insert_batches(erp_chars)
else:
    log.info("No characteristics created.")

# 2 - Sync Indicator Positions with APM

* Indicator positions are derived from the *APM indicator position*.

* The migration tool proposes the APM indicator positions, but user has an option to override the proposal by providing an input.

* The view `V_APMIndicatorPositions` contains the list of unique indicator positions derived from above.

* Indicator positions are **mandatory** to be present (or) created in the APM system before migrating the indicators.

**Process:**
1. We check the APM system `[GET]` for indicator positions, based on the position name.

2. If the value already exists in the APM system, we derive the `apm_guid` of the indicator position from the system.

3. In case if the record does not exist, a new indicator position is created `[POST]` in the APM system.

In [ ]:
import pandas as pd
from modules.apm.manage_indicators import ApiIndicatorPosition
from modules.util.database import APM_IndicatorPositions

Logger.blank_line(log)
log.info("Load: APM Indicator Positions")
Logger.blank_line(log)

file = rf'{EXTRACTION_DIR}/2_APM_Indicator_Positions.parquet'

api_ind_pos = ApiIndicatorPosition(CONFIG_ID)

indicator_positions = api_ind_pos.get_indicator_positions()
df = pd.json_normalize(indicator_positions, sep='_')
df.to_parquet(file, index=False)
log.info(f'{file} generated')

if db.drop_reload:
    db.truncate(APM_IndicatorPositions) # truncate all existing data (remove if needed)

positions = SQLAlchemyClient.dataframe_to_object(df, APM_IndicatorPositions)
if positions:
    db.insert_batches(positions)

In [ ]:
# ------------------------------------------------------------------------------ #
# Create missing indicator positions
# ------------------------------------------------------------------------------ #

# standard imports
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# custom imports
from modules.util.api import APIException
from modules.util.database import V_APMIndicatorPositions, APM_IndicatorPositions
from modules.apm.manage_indicators import ApiIndicatorPosition
from modules.util.helpers import convert_dataframe

# ------------------------------------------------------------------------------ #

api = ApiIndicatorPosition(CONFIG_ID)

results = db.select(
    model = V_APMIndicatorPositions,
    where = [V_APMIndicatorPositions.apm_guid == None]
)

# process the missing indicator positions
if len(results) > 0:
    df = pd.DataFrame(results)
    
    # remove duplicates (there can be multiple indicators with the same position)
    df_missing_position = df.drop_duplicates(subset=['APMIndicatorPosition'])
    log.info(f"{len(df_missing_position)} indicator positions to be created")

    if len(df_missing_position) > 0:
        api = ApiIndicatorPosition(CONFIG_ID)
        response = []
        error = []

        def call_api(id):
            return(api.create_indicator_position(id))
        
        with ThreadPoolExecutor(max_workers=20) as executor:
            future_char = {executor.submit(call_api, id): id for id in df_missing_position['APMIndicatorPosition']}

            for future in as_completed(future_char):
                id = future_char[future]
                try:
                    data = future.result()
                    response.append(data)
                except APIException as api_e:
                    
                    # 409: indicator position already exists
                    if api_e.status_code == 409:
                        try:
                            res = api.get_indicator_position_name(id)
                            response.append(res)
                        except APIException as api_e:
                            log.error(f"Exception for Indicator Position: {id}")
                            err = {
                                'id': id,
                                'status_code': api_e.status_code,
                                'response': api_e.response,
                                'endpoint': api_e.endpoint
                            }
                            error.append(err)
                    else:
                        log.error(f"Exception for Indicator Position: {id}")
                        err = {
                            'id': id,
                            'status_code': api_e.status_code,
                            'response': api_e.response,
                            'endpoint': api_e.endpoint
                        }
                        error.append(err)
        
        log.info(f"{len(response)} indicator positions created.")
        log.info(f"{len(error)} indicator positions had errors.")

        if response:
            df_ind_positions = pd.json_normalize(response, sep='_')
            cols = ['ID', 'SSID', 'name']
            df_ind_positions = df_ind_positions[cols]
            df_ind_positions = convert_dataframe(df_ind_positions)

            log.info(f"Updating {APM_IndicatorPositions.__tablename__}")
            positions = SQLAlchemyClient.dataframe_to_object(df_ind_positions, APM_IndicatorPositions)
            db.insert_batches(positions)
            log.info(f"Updated {APM_IndicatorPositions.__tablename__} with {db.count(APM_IndicatorPositions)} records")
        
        if error:
            df_error = pd.DataFrame(error)
            file_err = rf"{REPORTS_DIR}/3_APM_Indicator_Positions_Errors.csv"
            df_error.to_csv(file_err, index=False)
            log.error(f"{file_err} generated.")
else:
    log.warning(f"No indicator positions to be created.")